# Smart Travel App 
## Notebook 2: Data Read In

##### Group 4, Project 4 
##### Team: Dominique Villarreal, Enrique Garcia, Jose Santos
##### Project Due Date: June 12, 2023

In [1]:
#pip install --upgrade tensorflow
#import tensorflow as tf
#print(tf.__version__)

In [2]:
import json
import os
import pandas as pd
import pprint as pp
import requests
import time
#from tensorflow.keras.models import load_model
import warnings
warnings.simplefilter("ignore")
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [3]:
# Import csv's & load df's
# Denver
#merged_file_path = '/Users/josesantos/LearnPython/BCGit/G4_Project4/Resources/df_Extractions/merged_df.csv'
merged_file_path = '../G4_Project4/Resources/df_Extractions/merged_df.csv'
merged_df = pd.read_csv(merged_file_path)

merged_df.head(2)

,categories,name,rating,review_count,location,coordinates,city
0,"['Festivals', 'Arts & Crafts', 'Local Flavor']",Jackalope Arts,5.0,8,"{'address1': '5738 Olde Wadsworth Blvd', 'addr...","{'latitude': 39.800724136818026, 'longitude': ...",Denver
1,"['Venues & Event Spaces', 'Party & Event Plann...",Special Occasions Events,5.0,4,"{'address1': '3550 Federal Blvd', 'address2': ...","{'latitude': 39.76623, 'longitude': -105.02439}",Denver


In [4]:
# Process & Organize merged_df
columns_to_delete = ['location', 'coordinates']
merged_df.drop(columns_to_delete, axis=1, inplace=True)

merged_df = merged_df[['city','name', 'rating', 
                       'review_count', 'categories']]

# Access and work with the updated DataFrame
merged_df.head(1)

,city,name,rating,review_count,categories
0,Denver,Jackalope Arts,5.0,8,"['Festivals', 'Arts & Crafts', 'Local Flavor']"


In [5]:
# City - One-hot encoding
#city_encoded_data = pd.get_dummies(merged_df, columns = ['city'])
#city_encoded_data

merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19627 entries, 0 to 19626
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   city          19627 non-null  object 
 1   name          19627 non-null  object 
 2   rating        19627 non-null  float64
 3   review_count  19627 non-null  int64  
 4   categories    19627 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 766.8+ KB


In [6]:
# Create encoded df
city_encoded_data = merged_df.copy()
city_encoded_data

,city,name,rating,review_count,categories
0,Denver,Jackalope Arts,5.0,8,"['Festivals', 'Arts & Crafts', 'Local Flavor']"
1,Denver,Special Occasions Events,5.0,4,"['Venues & Event Spaces', 'Party & Event Plann..."
2,Denver,Denver International Wine Festival,4.5,6,['Festivals']
3,Denver,"Denver Mineral, Fossil, Gem & Jewelry Show",4.5,10,['Festivals']
4,Denver,Colorado Festival of Horror,5.0,1,['Festivals']
...,...,...,...,...,...
19622,NewYork,Felice 56,4.0,94,"['Italian', 'Wine Bars']"
19623,NewYork,The Jeffrey Craft Beer & Bites,4.0,728,"['Bars', 'American (New)', 'Coffee & Tea']"
19624,NewYork,Saint Tuesday,4.0,30,"['Lounges', 'Speakeasies', 'Cocktail Bars']"
19625,NewYork,Harlem Nights Bar,4.0,185,"['Bars', 'Music Venues']"


In [7]:
# Explode the 'categories' column
city_encoded_data['categories'] = city_encoded_data['categories'].apply(eval)
city_encoded_data = city_encoded_data.explode('categories')
city_encoded_data

,city,name,rating,review_count,categories
0,Denver,Jackalope Arts,5.0,8,Festivals
0,Denver,Jackalope Arts,5.0,8,Arts & Crafts
0,Denver,Jackalope Arts,5.0,8,Local Flavor
1,Denver,Special Occasions Events,5.0,4,Venues & Event Spaces
1,Denver,Special Occasions Events,5.0,4,Party & Event Planning
...,...,...,...,...,...
19625,NewYork,Harlem Nights Bar,4.0,185,Bars
19625,NewYork,Harlem Nights Bar,4.0,185,Music Venues
19626,NewYork,Cardiff Giant,4.0,56,Bars
19626,NewYork,Cardiff Giant,4.0,56,"Beer, Wine & Spirits"


In [8]:
city_encoded_data.to_csv('city_encoded_data.csv', index=False)

In [9]:
import pandas as pd

# Assuming your DataFrame is named "city_encoded_data"

# Filter the DataFrame for rows with the city "Denver"
denver_data = city_encoded_data[city_encoded_data['city'] == 'Denver']

# Get the unique categories for Denver
denver_unique_categories = denver_data['categories'].unique()

# Filter the DataFrame for rows with cities other than "Denver"
other_cities_data = city_encoded_data[city_encoded_data['city'] != 'Denver']

# Get the unique categories for the other cities
other_cities_unique_categories = other_cities_data['categories'].unique()

# Get the categories that are unique to Denver
unique_to_denver_categories = set(denver_unique_categories) - set(other_cities_unique_categories)

# Create a DataFrame with the unique categories for Denver
denver_unique_categories_df = pd.DataFrame({'Unique Categories for Denver': list(unique_to_denver_categories)})

# Add city column
denver_unique_categories_df['city'] = 'Denver'

# Rename unique categories column
denver_unique_categories_df['Unique Categories'] = denver_unique_categories_df['Unique Categories for Denver']

# Drop Unique Categories for New York
denver_unique_categories_df.drop('Unique Categories for Denver', axis=1, inplace=True)

# Print the DataFrame
denver_unique_categories_df


,city,Unique Categories
0,Denver,Sports Betting
1,Denver,Hobby Shops
2,Denver,Party Supplies
3,Denver,Rugs
4,Denver,Water Delivery
...,...,...
66,Denver,Eritrean
67,Denver,Art Supplies
68,Denver,Pet Adoption
69,Denver,Metal Fabricators


In [10]:
import pandas as pd

# Assuming your DataFrame is named "city_encoded_data"

# Filter the DataFrame for rows with the city "Miami"
miami_data = city_encoded_data[city_encoded_data['city'] == 'Miami']

# Get the unique categories for Miami
miami_unique_categories = miami_data['categories'].unique()

# Filter the DataFrame for rows with cities other than "Miami"
other_cities_data = city_encoded_data[city_encoded_data['city'] != 'Miami']

# Get the unique categories for the other cities
other_cities_unique_categories = other_cities_data['categories'].unique()

# Get the categories that are unique to Miami
unique_to_miami_categories = set(miami_unique_categories) - set(other_cities_unique_categories)

# Create a DataFrame with the unique categories for Miami
miami_unique_categories_df = pd.DataFrame({'Unique Categories for Miami': list(unique_to_miami_categories)})

# Add city column
miami_unique_categories_df['city'] = 'Miami'

# Rename unique categories column
miami_unique_categories_df['Unique Categories'] = miami_unique_categories_df['Unique Categories for Miami']

# Drop Unique Categories for New York
miami_unique_categories_df.drop('Unique Categories for Miami', axis=1, inplace=True)

# Print the DataFrame
miami_unique_categories_df


,city,Unique Categories
0,Miami,Motorcycle Dealers
1,Miami,Basque
2,Miami,Local Fish Stores
3,Miami,Hotels & Travel
4,Miami,Golf Cart Rentals
...,...,...
64,Miami,Diving
65,Miami,Resorts
66,Miami,Water Parks
67,Miami,Vacation Rentals


In [11]:
import pandas as pd

# Assuming your DataFrame is named "city_encoded_data"

# Filter the DataFrame for rows with the city "NewYork"
newyork_data = city_encoded_data[city_encoded_data['city'] == 'NewYork']

# Get the unique categories for New York, dropping NaN values
newyork_unique_categories = newyork_data['categories'].dropna().unique()

# Filter the DataFrame for rows with cities other than "NewYork"
other_cities_data = city_encoded_data[city_encoded_data['city'] != 'NewYork']

# Get the unique categories for the other cities, dropping NaN values
other_cities_unique_categories = other_cities_data['categories'].dropna().unique()

# Get the categories that are unique to New York
unique_to_newyork_categories = set(newyork_unique_categories) - set(other_cities_unique_categories)

# Create a DataFrame with the unique categories for New York
newyork_unique_categories_df = pd.DataFrame({'Unique Categories for New York': list(unique_to_newyork_categories)})

# Add city column
newyork_unique_categories_df['city'] = 'New York'

# Rename unique categories column
newyork_unique_categories_df['Unique Categories'] = newyork_unique_categories_df['Unique Categories for New York']

# Drop Unique Categories for New York
newyork_unique_categories_df.drop('Unique Categories for New York', axis=1, inplace=True)

# Print the DataFrame
newyork_unique_categories_df


,city,Unique Categories
0,New York,Scottish
1,New York,Hainan
2,New York,Taxis
3,New York,Japanese Curry
4,New York,Language Schools
5,New York,Adult Education
6,New York,Florists
7,New York,Pest Control
8,New York,Fondue
9,New York,Public Transportation


In [12]:
# Concatenate the DataFrames
combined_unique_df = pd.concat([newyork_unique_categories_df, miami_unique_categories_df, denver_unique_categories_df], ignore_index=True)

# Print the combined DataFrame
combined_unique_df

# Export to csv for review
combined_unique_df.to_csv('combined_unique_categories.csv', index=False)


In [17]:
# Call in the selected unique df
# Import csv's & load df's
# Denver
selected_unique_categories_file_path = '../G4_Project4/Resources/selected_unique_categories.csv'
selected_unique_cat_df = pd.read_csv(selected_unique_categories_file_path)

selected_unique_cat_df.head(200)

,city,Unique Categories
0,New York,Fondue
1,New York,Sicilian
2,New York,Club Crawl
3,New York,Badminton
4,New York,Ski Resorts
...,...,...
109,Denver,Ski & Snowboard Shops
110,Denver,Pumpkin Patches
111,Denver,Ski Schools
112,Denver,Somali


In [18]:
# Categories - labelEncoder
# Create a label encoder
label_encoder = LabelEncoder()

# Apply label encoding to the 'categories' column
city_encoded_data['categories_encoded'] = label_encoder.fit_transform(city_encoded_data['categories'])

city_encoded_data

,city,name,rating,review_count,categories,categories_encoded
0,Denver,Jackalope Arts,5.0,8,Festivals,220
0,Denver,Jackalope Arts,5.0,8,Arts & Crafts,38
0,Denver,Jackalope Arts,5.0,8,Local Flavor,348
1,Denver,Special Occasions Events,5.0,4,Venues & Event Spaces,605
1,Denver,Special Occasions Events,5.0,4,Party & Event Planning,416
...,...,...,...,...,...,...
19625,NewYork,Harlem Nights Bar,4.0,185,Bars,58
19625,NewYork,Harlem Nights Bar,4.0,185,Music Venues,384
19626,NewYork,Cardiff Giant,4.0,56,Bars,58
19626,NewYork,Cardiff Giant,4.0,56,"Beer, Wine & Spirits",70


In [19]:
filtered_city_data = city_encoded_data[city_encoded_data['categories'].isin(selected_unique_cat_df['Unique Categories'])]
filtered_city_df = filtered_city_data
filtered_city_df.head(500)

,city,name,rating,review_count,categories,categories_encoded
0,Denver,Jackalope Arts,5.0,8,Local Flavor,348
7,Denver,Cherry Creek Holiday Market,3.5,17,Public Markets,467
20,Denver,The Brew Hut,5.0,121,Brewing Supplies,97
39,Denver,Altitude Brewing & Supply,4.5,16,Brewing Supplies,97
234,Denver,Body Balance Strength & Wellness,5.0,6,Chiropractors,142
...,...,...,...,...,...,...
19413,NewYork,The Lavaux,4.5,100,Fondue,231
19536,NewYork,Skillman Music Recording Studio,5.0,2,Music Production Services,383
19578,NewYork,Mamak House,4.0,133,Malaysian,354
19592,NewYork,Amor y Amargo,4.0,507,Vermouth Bars,606


In [20]:
# Drop original categories column
#filtered_city_df.drop('categories', axis=1, inplace=True)

In [21]:
# Create the city_category_predict_df
city_category_predict_df = filtered_city_df[['city', 'categories','categories_encoded']]
city_category_predict_df

,city,categories,categories_encoded
0,Denver,Local Flavor,348
7,Denver,Public Markets,467
20,Denver,Brewing Supplies,97
39,Denver,Brewing Supplies,97
234,Denver,Chiropractors,142
...,...,...,...
19413,NewYork,Fondue,231
19536,NewYork,Music Production Services,383
19578,NewYork,Malaysian,354
19592,NewYork,Vermouth Bars,606


In [24]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import random

# select X & y and reshape df
X = city_category_predict_df['categories_encoded'].values.reshape(-1, 1)
y = city_category_predict_df['city']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the Random Forest Classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Set to keep track of predicted cities
predicted_cities = set()

# Loop until Denver, Miami, and NewYork have been predicted
while len(predicted_cities) < 3:
    # Generate a random sample of 10 categories
    sample_categories = random.sample(list(city_category_predict_df['categories_encoded']), 10)
    sample_categories_encoded = pd.DataFrame({'categories_encoded': sample_categories})

    # Predict the city based on the random group of 10 categories
    prediction = model.predict(sample_categories_encoded)
    predicted_city = prediction[0]

    # Check if the predicted city is one of Denver, Miami, or NewYork
    if predicted_city in ['Denver', 'Miami', 'NewYork']:
        # Add the predicted city to the set of predicted cities
        predicted_cities.add(predicted_city)

        # Get the actual category names
        category_names = city_category_predict_df.loc[city_category_predict_df['categories_encoded'].isin(sample_categories), 'categories'].unique()

        # Print the categories used to predict the city
        print("Categories Used:")
        for category in category_names:
            print(category)

        # Print the predicted city
        print("Predicted City:", predicted_city)
        print()


Categories Used:
Chiropractors
Hobby Shops
Art Consultants
Eyelash Service
Champagne Bars
Moroccan
Malaysian
Buddhist Temples
Pedicabs
Club Crawl
Predicted City: Denver

Categories Used:
Somali
Barbers
Video Game Stores
Costumes
Art Consultants
Herbal Shops
Skate Shops
Beach Equipment Rentals
Moroccan
Fondue
Predicted City: NewYork

Categories Used:
Brewing Supplies
Gemstones & Minerals
Piercing
Herbal Shops
Snorkeling
Skydiving
Water Parks
Georgian
Malaysian
Predicted City: NewYork

Categories Used:
Sledding
Olive Oil
Ski & Snowboard Shops
Bingo Halls
Books, Mags, Music & Video
Golf Cart Rentals
Kiteboarding
Skate Shops
Scooter Rentals
Parasailing
Predicted City: Denver

Categories Used:
Local Flavor
Used Bookstore
Flea Markets
Herbal Shops
Skydiving
Beach Equipment Rentals
Boat Dealers
Sicilian
South African
Language Schools
Predicted City: Denver

Categories Used:
Used Bookstore
Eritrean
Furniture Stores
Parasailing
Nicaraguan
Moroccan
Georgian
Malaysian
South African
Sommelier Serv

In [ ]:
model.score(X_test, y_test)

In [ ]:
model.fit(X_test, y_test)

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import random

# select X & y and reshape df
X = city_category_predict_df['categories_encoded'].values.reshape(-1, 1)
y = city_category_predict_df['city']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the Random Forest Classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Set to keep track of predicted cities
predicted_cities = set()

# Loop until Denver, Miami, and NewYork have been predicted
while len(predicted_cities) < 3:
    # Generate a random sample of 10 categories
    sample_categories = random.sample(list(city_category_predict_df['categories_encoded']), 10)
    sample_categories_encoded = pd.DataFrame({'categories_encoded': sample_categories})

    # Predict the city based on the random group of 10 categories
    prediction = model.predict(sample_categories_encoded)
    predicted_city = prediction[0]

    # Check if the predicted city is one of Denver, Miami, or NewYork
    if predicted_city in ['Denver', 'Miami', 'NewYork']:
        # Add the predicted city to the set of predicted cities
        predicted_cities.add(predicted_city)

        # Get the actual category names
        category_names = city_category_predict_df.loc[city_category_predict_df['categories_encoded'].isin(sample_categories), 'categories'].unique()

        # Print the categories used to predict the city
        print("Categories Used:")
        for category in category_names:
            print(category)

        # Print the predicted city
        print("Predicted City:", predicted_city)

        # Print the top 3 activities for each sample category
        print("Top Activities:")
        for category in sample_categories:
            filtered_activities = filtered_city_df[(filtered_city_df['city'] == predicted_city) & (filtered_city_df['categories_encoded'] == category)]
            sorted_activities = filtered_activities.sort_values(['rating', 'review_count'], ascending=[False, False])
            top_activities = sorted_activities['name'].head(3).tolist()

            print("Category:", category)
            for activity in top_activities:
                print(activity)

        print()


In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import random

# select X & y and reshape df
X = city_category_predict_df['categories_encoded'].values.reshape(-1, 1)
y = city_category_predict_df['city']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the Random Forest Classifier
model2 = RandomForestClassifier(n_estimators=100, random_state=42)
model2.fit(X_train, y_train)

# Set to keep track of predicted cities
predicted_cities = set()

# Loop until Denver, Miami, and NewYork have been predicted
while len(predicted_cities) < 3:
    # Generate a random sample of 10 categories
    sample_categories = random.sample(list(city_category_predict_df['categories_encoded']), 10)
    sample_categories_encoded = pd.DataFrame({'categories_encoded': sample_categories})

    # Predict the city based on the random group of 10 categories
    prediction = model.predict(sample_categories_encoded)
    predicted_city = prediction[0]

    # Check if the predicted city is one of Denver, Miami, or NewYork
    if predicted_city in ['Denver', 'Miami', 'NewYork']:
        # Add the predicted city to the set of predicted cities
        predicted_cities.add(predicted_city)

        # Get the actual category names
        category_names = city_category_predict_df.loc[city_category_predict_df['categories_encoded'].isin(sample_categories), 'categories'].unique()

        # Print the categories used to predict the city
        print("Categories Used:")
        for category in category_names:
            print(category)

        # Print the predicted city
        print("Predicted City:", predicted_city)

        # Print the top activities for each sample category
        print("Top Activities:")
        for category in sample_categories:
            # Convert category back to its name
            category_name = city_category_predict_df.loc[city_category_predict_df['categories_encoded'] == category, 'categories'].iloc[0]

            filtered_activities = filtered_city_df[(filtered_city_df['city'] == predicted_city) & (filtered_city_df['categories_encoded'] == category)]
            sorted_activities = filtered_activities.sort_values(['rating', 'review_count'], ascending=[False, False])
            top_activities = sorted_activities[['name', 'rating', 'review_count']].head(3)

            print("Category:", category_name)
            for _, activity in top_activities.iterrows():
                activity_name = activity['name']
                rating = activity['rating']
                review_count = activity['review_count']
                print("Activity:", activity_name)
                print("Rating:", rating)
                print("Review Count:", review_count)
                print()

        print()


In [ ]:
model2.score(X_test, y_test)

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import random

# select X & y and reshape df
X = city_category_predict_df['categories_encoded'].values.reshape(-1, 1)
y = city_category_predict_df['city']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the Random Forest Classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Set to keep track of predicted cities and their activities
predicted_data = []

# Loop until Denver, Miami, and NewYork have been predicted
while len(predicted_data) < 3:
    # Generate a random sample of 10 categories
    sample_categories = random.sample(list(city_category_predict_df['categories_encoded']), 10)
    sample_categories_encoded = pd.DataFrame({'categories_encoded': sample_categories})

    # Predict the city based on the random group of 10 categories
    prediction = model.predict(sample_categories_encoded)
    predicted_city = prediction[0]

    # Check if the predicted city is one of Denver, Miami, or NewYork
    if predicted_city in ['Denver', 'Miami', 'NewYork']:
        # Get the actual category names
        category_names = city_category_predict_df.loc[city_category_predict_df['categories_encoded'].isin(sample_categories), 'categories'].unique()

        # Get the top activities for each sample category
        activities_data = []
        for category in sample_categories:
            # Convert category back to its name
            category_name = city_category_predict_df.loc[city_category_predict_df['categories_encoded'] == category, 'categories'].iloc[0]

            filtered_activities = filtered_city_df[(filtered_city_df['city'] == predicted_city) & (filtered_city_df['categories_encoded'] == category)]
            sorted_activities = filtered_activities.sort_values(['rating', 'review_count'], ascending=[False, False])
            top_activities = sorted_activities[['name', 'rating', 'review_count']].head(3)

            for _, activity in top_activities.iterrows():
                activity_name = activity['name']
                rating = activity['rating']
                review_count = activity['review_count']

                activities_data.append({
                    'Sample Category': category_name,
                    'Predicted City': predicted_city,
                    'Activity Name': activity_name,
                    'Rating': rating,
                    'Review Count': review_count
                })

        # Append the predicted activities to the list
        predicted_data.extend(activities_data)

# Create a DataFrame from the predicted data
predicted_df = pd.DataFrame(predicted_data)

# Print the final predicted data DataFrame
print("Predicted Data:")
print(predicted_df)


In [ ]:
predicted_df